In [ ]:
import pandas as pd

In [ ]:
test_path = "/home/monlamai/Documents/GitHub/combining-dataset/550-train.tsv"
df = pd.read_csv(test_path, sep='\t')

In [ ]:
df.head()

In [ ]:
df = df[df['dept'] == 'STT_TT']
df['len'] = df['uni'].apply(lambda x: len(x))

In [ ]:
df.head()

In [ ]:
df.sort_values(by=['len'], inplace=True, ascending=False, ignore_index=True)

In [ ]:
# 60005
# 70021
# i = len(df) - 70021
# i = 2222
i = 2431
df.loc[i, 'url'], df.loc[i, 'uni']

In [ ]:
from transformers import pipeline
import pyewts
generator = pipeline(model="openpecha/wav2vec2_run8")
# spsither/whipser-small-r2
# generator = pipeline(model="spsither/whipser-small-r2")

opt = generator(df.loc[i, 'url'])
inf = opt['text']
print(inf)

generator = pipeline(model="TenzinGayche/whisper-small-3")
# generator = pipeline(model="spsither/whipser-small-r2")
opt = generator(df.loc[i, 'url'])
inf = opt['text']

converter = pyewts.pyewts()
print(converter.toUnicode(inf))

generator = pipeline(model="spsither/whipser-small-r2")
opt = generator(df.loc[i, 'url'])
inf = opt['text']

converter = pyewts.pyewts()
print(converter.toUnicode(inf))

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import Audio, load_dataset

# load model and processor
processor = WhisperProcessor.from_pretrained("spsither/whipser-small-r2")
model = WhisperForConditionalGeneration.from_pretrained("spsither/whipser-small-r2")
forced_decoder_ids = processor.get_decoder_prompt_ids(language="tibetan", task="transcribe")

print(forced_decoder_ids)

In [ ]:
from datasets import Dataset

ds = Dataset.from_pandas(df)
ds = ds.cast_column("url", Audio(sampling_rate=16_000))
input_speech = next(iter(ds))["url"]

print(input_speech["array"], input_speech["sampling_rate"])

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import Audio, load_dataset

# load model and processor
processor = WhisperProcessor.from_pretrained("spsither/whipser-small-r2")
model = WhisperForConditionalGeneration.from_pretrained("spsither/whipser-small-r2")
forced_decoder_ids = processor.get_decoder_prompt_ids(language="tibetan", task="transcribe")

model.generation_config.no_repeat_ngram_size = 2

# model.generation_config.num_beams = 1 # default
model.generation_config.num_beams = 3
model.generation_config.exponential_decay_length_penalty = 1
# load streaming dataset and read first audio sample
from datasets import Dataset

ds = Dataset.from_pandas(df)
ds = ds.cast_column("url", Audio(sampling_rate=16_000))
# input_speech = next(iter(ds))["url"]
imput_speech = ds[i]["url"]

input_features = processor(input_speech["array"], sampling_rate=input_speech["sampling_rate"], return_tensors="pt").input_features

# generate token ids
predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
# decode token ids to text
# transcription = processor.batch_decode(predicted_ids)
# print(transcription)

transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
converter = pyewts.pyewts()
print(converter.toUnicode(transcription[0]))

In [ ]:
# import os
from transformers import pipeline
from datasets import Dataset

generator = pipeline(model="openpecha/wav2vec2_run8", device="cuda:0")


# opt = generator(df.loc[0, 'url'])
# inf = opt['text']
# print(inf)

dataset = Dataset.from_pandas(df)

def generate(batch):
    urls = batch["url"]
    generated_texts = generator(urls)  # This should process all URLs in the batch at once
    # Extracting the 'text' field from each generated output and assigning it back to the batch
    batch['inf'] = [item['text'] for item in generated_texts]
    return batch

# df['inf']  = df.apply(lambda row: generator(row['url'])['text'], axis=1)
# updated_dataset = dataset.map(generate, batched=True, batch_size=8)

In [ ]:
df_out = updated_dataset.to_pandas()

In [ ]:
# import pyewts

# converter = pyewts.pyewts()

# df_out['inf_uni'] = df_out['inf'].map(lambda x : converter.toUnicode(x))

In [ ]:
from evaluate import load

cer_metric = load("cer")

In [ ]:
df_out

cer = cer_metric.compute(references=[df_out.loc[0, 'uni']], predictions=[df_out.loc[0, 'inf']])

print(cer)

In [ ]:
df_out['cer'] = df_out.apply(lambda row: min(cer_metric.compute(references=[df_out.loc[0, 'uni']], predictions=[df_out.loc[0, 'inf']]), 1), axis=1)

In [ ]:
mean_cer = df_out['cer'].mean()

In [ ]:
mean_cer

In [ ]:
mean_cer * 100

In [ ]:
df_out.head()

In [ ]:

df_out.to_csv('test_wav2vec2.tsv', sep='\t', index=False)